In [1]:
import numpy as np
import pandas as pd
import itertools
import larch
import os
import re
import yaml

from pytest import approx
from itertools import combinations

# Estimating Coordinated Daily Activity Patterns

This notebook illustrates how to re-estimate ActivitySim's CDAP model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - TODO: Update the ActivitySim model coefficients and re-run the model in simulation mode.
  

In [2]:
from larch.util.activitysim import cdap_model

In [3]:
# %cd test
# !activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

os.chdir(os.path.expanduser("~/sandbox/cdap-est/test_example_estimation"))
# TODO: integrate with command above

In [4]:
edb_directory = "output/estimation_data_bundle/cdap/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

def read_yaml(filename, **kwargs):
    with open(os.path.join(edb_directory, filename), 'rt') as f:
        return yaml.load(f, Loader=yaml.SafeLoader, **kwargs)

# Load Inputs

## Settings

In [5]:
settings = read_yaml("cdap_model_settings.yaml")
settings

{'INDIV_AND_HHSIZE1_SPEC': 'cdap_indiv_and_hhsize1.csv',
 'FIXED_RELATIVE_PROPORTIONS_SPEC': 'cdap_fixed_relative_proportions.csv',
 'CONSTANTS': {'FULL': 1,
  'PART': 2,
  'UNIVERSITY': 3,
  'NONWORK': 4,
  'RETIRED': 5,
  'DRIVING': 6,
  'SCHOOL': 7,
  'PRESCHOOL': 8},
 'annotate_persons': {'SPEC': 'annotate_persons_cdap', 'DF': 'persons'},
 'annotate_households': {'SPEC': 'annotate_households_cdap',
  'DF': 'households',
  'TABLES': ['persons']}}

## Households

In [6]:
hhs = pd.read_csv("./data_test/households.csv")
hhs

,household_id,TAZ,income,hhsize,HHT,auto_ownership,num_workers
0,982875,16,30900,2,5,1,2
1,1810015,16,99700,9,2,1,4
2,1099626,20,58160,3,1,1,1
3,763879,6,59220,1,4,1,0
4,824207,18,51000,1,4,0,1
...,...,...,...,...,...,...,...
1995,1444704,14,72000,1,4,1,1
1996,2704362,25,128600,2,1,0,0
1997,109427,10,16000,1,4,0,1
1998,2820275,8,0,1,0,0,1


## Persons

In [7]:
persons = pd.read_csv("./data_test/persons.csv")
persons

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype
0,25675,25675,27,1,2,3,2,3
1,25678,25678,30,1,2,3,3,4
2,25683,25683,23,1,1,3,3,4
3,25684,25684,52,1,1,3,3,4
4,25691,25691,19,1,1,3,2,3
...,...,...,...,...,...,...,...,...
3332,7540037,2848702,20,1,1,3,3,4
3333,7554789,2863454,68,1,1,3,3,5
3334,7554829,2863494,93,1,2,3,3,5
3335,7554833,2863498,82,1,2,3,3,5


In [8]:
from activitysim.abm.models.util.cdap import assign_cdap_rank
person_rank = assign_cdap_rank(persons)

## interaction_coef

In [9]:
interaction_coef = read_csv(
    "cdap_interaction_coefficients.csv",
    dtype={
        'interaction_ptypes':str,
    },
    keep_default_na=False,
)
interaction_coef

,activity,interaction_ptypes,coefficient,cardinality,slug
0,H,11,1.6260,2,HH11
1,H,12,0.7407,2,HH12
2,H,13,1.1830,2,HH13
3,H,14,0.9436,2,HH14
4,H,15,1.2980,2,HH15
...,...,...,...,...,...
128,N,,-1.3460,4,NNNN
129,H,,-3.7330,4,HHHH
130,M,,-1.5280,5,MMMMM
131,N,,-3.4530,5,NNNNN


## Spec 1

In [10]:
spec1 = read_csv("cdap_INDIV_AND_HHSIZE1_SPEC.csv", comment='#')
spec1.head()

,Description,Expression,M,N,H
0,Full-time worker alternative-specific constants,ptype == 1,1.378735,0.622662,NaN
1,Part-time worker alternative-specific constants,ptype == 2,-0.718824,0.636032,NaN
2,University student alternative-specific constants,ptype == 3,2.353595,0.609710,NaN
3,Non-working adult alternative-specific constants,ptype == 4,-999.000000,0.594645,NaN
4,Retired alternative-specific constants,ptype == 5,-999.000000,0.408202,NaN


## Values

In [12]:
values = read_csv("cdap_values_combined.csv", comment='#')
values.head()

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
0,25675,M,M,25675,27,1,2,3,2,3,...,314.01431,0,0.0,72.14684,0.0,1,5.52555,38.187500,978.875,36.753679
1,25678,N,N,25678,30,1,2,3,3,4,...,546.00000,0,0.0,0.00000,0.0,1,5.00004,149.333333,252.600,93.850390
2,25683,N,N,25683,23,1,1,3,3,4,...,546.00000,0,0.0,0.00000,0.0,1,5.00004,149.333333,252.600,93.850390
3,25684,N,N,25684,52,1,1,3,3,4,...,546.00000,0,0.0,0.00000,0.0,1,5.00004,149.333333,252.600,93.850390
4,25691,M,M,25691,19,1,1,3,2,3,...,546.00000,0,0.0,0.00000,0.0,1,5.00004,149.333333,252.600,93.850390


In [13]:
values['cdap_rank'] = persons['cdap_rank']

# Model

In [18]:
model = cdap_model(hhs, values, spec1, interaction_coef)

one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>
one-hot encoding choice array
converting data_ch to <class 'numpy.float64'>


In [22]:
model[0].utility_functions() ## HH Size = 1 

alt,formula
1,+ P.coef_000_M * X('ptype == 1') + P.coef_001_M * X('ptype == 2') + P.coef_002_M * X('ptype == 3') + P.coef_003_M * X('ptype == 4') + P.coef_004_M * X('ptype == 5') + P.coef_005_M * X('ptype == 6') + P.coef_006_M * X('ptype == 7') + P.coef_007_M * X('(ptype == 7) & (age >= 6) & (age <= 9)') + P.coef_008_M * X('(ptype == 7) & (age >= 13) & (age <= 15)') + P.coef_009_M * X('ptype == 8') + P.coef_010_M * X('(ptype == 8) & (age >= 0) & (age <= 1)') + P.coef_011_M * X('(ptype == 8) & (age >= 4) & (age <= 5)') + P.coef_012_M * X('(ptype == 1) & (age < 40)') + P.coef_014_M * X('(ptype == 1) & (sex == 2)') + P.coef_015_M * X('(ptype == 4) & (sex == 2)') + P.coef_016_M * X('(ptype == 5) & (sex == 2)') + P.coef_017_M * X('(ptype == 4) & (auto_ownership > num_workers)') + P.coef_018_M * X('(ptype == 5) & (auto_ownership > num_workers)') + P.coef_038_M * X('(ptype == 1) * auPkTotal') + P.coef_039_M * X('(ptype == 2) * auPkTotal') + P.coef_040_M * X('(ptype == 4) * auPkTotal') + P.coef_041_M * X('(ptype == 5) * auPkTotal')
2,+ P.coef_000_N * X('ptype == 1') + P.coef_001_N * X('ptype == 2') + P.coef_002_N * X('ptype == 3') + P.coef_003_N * X('ptype == 4') + P.coef_004_N * X('ptype == 5') + P.coef_005_N * X('ptype == 6') + P.coef_006_N * X('ptype == 7') + P.coef_008_N * X('(ptype == 7) & (age >= 13) & (age <= 15)') + P.coef_009_N * X('ptype == 8') + P.coef_017_N * X('(ptype == 4) & (auto_ownership > num_workers)') + P.coef_018_N * X('(ptype == 5) & (auto_ownership > num_workers)') + P.coef_019_N * X('(ptype == 8) & (auto_ownership > num_workers)') + P.coef_030_N * X('(ptype == 2) & (income_in_thousands >= 100)') + P.coef_042_N * X('(ptype == 4) * auOpRetail') + P.coef_043_N * X('(ptype == 5) * auOpRetail') + P.coef_044_N * X('(ptype == 3) * auOpRetail') + P.coef_045_N * X('(ptype == 6) * auOpRetail') + P.coef_046_N * X('(ptype == 7) * auOpRetail') + P.coef_047_N * X('(ptype == 8) * auOpRetail')
3,+ P.coef_013_H * X('(ptype == 5) & (age > 80)') + P.coef_020_H * X('(ptype == 1) & (auto_ownership < num_workers)') + P.coef_021_H * X('(ptype == 4) & (auto_ownership < num_workers)') + P.coef_022_H * X('(ptype == 5) & (auto_ownership < num_workers)') + P.coef_023_H * X('(ptype == 6) & (auto_ownership < num_workers)') + P.coef_024_H * X('(ptype == 7) & (auto_ownership < num_workers)') + P.coef_025_H * X('(ptype == 8) & (auto_ownership < num_workers)') + P.coef_026_H * X('(ptype == 1) & (income_in_thousands < 20)') + P.coef_027_H * X('(ptype == 5) & (income_in_thousands < 20)') + P.coef_028_H * X('(ptype == 2) & (income_in_thousands < 20)') + P.coef_029_H * X('(ptype == 2) & (income_in_thousands >= 50) & (income_in_thousands <= 100)') + P.coef_030_H * X('(ptype == 2) & (income_in_thousands >= 100)') + P.coef_031_H * X('(ptype == 4) & (income_in_thousands >= 50) & (income_in_thousands <= 100)') + P.coef_032_H * X('(ptype == 4) & (income_in_thousands >= 100)') + P.coef_033_H * X('(ptype == 6) & (income_in_thousands < 20)') + P.coef_034_H * X('(ptype == 6) & (income_in_thousands >= 50) & (income_in_thousands <= 100)') + P.coef_035_H * X('(ptype == 6) & (income_in_thousands >= 100)') + P.coef_036_H * X('(ptype == 8) & (income_in_thousands >= 50) & (income_in_thousands <= 100)') + P.coef_037_H * X('(ptype == 8) & (income_in_thousands >= 100)')


In [23]:
model[1].utility_functions() ## HH Size = 2

<xmle.Elem 'div' with 1 children>

In [19]:
model.estimate(method='SLSQP', options={'maxiter':1000})

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_000_M,25.476367,0.0,0.0,-inf,inf,0,,25.476367
coef_000_N,0.824959,0.0,0.0,-inf,inf,0,,0.824959
coef_001_M,11.503151,0.0,0.0,-inf,inf,0,,11.503151
coef_001_N,1.066949,0.0,0.0,-inf,inf,0,,1.066949
coef_002_M,2.699744,0.0,0.0,-inf,inf,0,,2.699744
...,...,...,...,...,...,...,...,...
interaction_128_NNNN,0.971650,0.0,0.0,-inf,inf,0,,0.971650
interaction_129_HHHH,0.122476,0.0,0.0,-inf,inf,0,,0.122476
interaction_130_MMMMM,0.889843,0.0,0.0,-inf,inf,0,,0.889843
interaction_131_NNNNN,-28.340923,0.0,0.0,-inf,inf,0,,-28.340923


<ipython-input-19-dbc5e2c47bd5>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate(method='SLSQP', options={'maxiter':1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 7.78562130340975e-43 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: coef_000_M               25.476367
┃                   coef_000_N                0.824959
┃                   coef_001_M               11.503151
┃                   coef_001_N                1.066949
┃                   coef_002_M                2.699744
┃                                              ...    
┃                   interaction_128_NNNN      0.971650
┃                   interaction_129_HHHH      0.122476
┃                   interaction_130_MMMMM     0.889843
┃                   interaction_131_NNNNN   -28.340923
┃                   interaction_132_HHHHH   -79.236045
┃                   Length: 193, dtype: float64
┣          loglike: -1830.6012870866707
┣        d_loglike: coef_000_M               5.532689e-05
┃                   coef_000_N              -1.119279e-04
┃                   coef_001_M               3.602897e-04
┃                   coef_001_N               1.164642e-04
┃                   coef_002_M               2.807719e-04
┃                                                ...     
┃                   interaction_128_NNNN     2.282432e-04
┃                   interaction_129_HHHH     3.351075e-04
┃                   interaction_130_MMMMM    9.632944e-05
┃                   interaction_131_NNNNN   -8.835538e-14
┃                   interaction_132_HHHHH   -9.031417e-33
┃                   Length: 193, dtype: float64
┣              nit: 159
┣             nfev: 267
┣             njev: 159
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=113, microseconds=605785)
┣           method: 'SLSQP'
┣          n_cases: 2000
┣ iteration_number: 159
┣          logloss: 0.9153006435433354

In [20]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value
coef_000_M,25.5,15.7,1.62,,NA,0.00
coef_000_N,0.825,0.321,2.57,*,NA,0.00
coef_001_M,11.5,22.6,0.51,,NA,0.00
coef_001_N,1.07,0.421,2.53,*,NA,0.00
coef_002_M,2.70,0.337,8.02,***,NA,0.00
coef_002_N,43.0,28.8,1.50,,NA,0.00
coef_003_M,-0.112,1.59e+04,-0.00,,NA,0.00
coef_003_N,-28.7,19.9,-1.44,,NA,0.00
coef_004_M,-0.0857,1.62e+04,-0.00,,NA,0.00
coef_004_N,8.99,17.8,0.51,,NA,0.00
